In [365]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [366]:
# set a seed
random.seed(123)
np.random.seed(123)

In [375]:
df=pd.read_csv("data/data.csv")

In [376]:
# extract data without null value and drop those meanless value
df_all = df.loc[df.shot_made_flag.notnull(),:]
df_all.drop('team_id',axis=1,inplace=True)
df_all.drop('team_name',axis=1,inplace=True)
df_all.drop('matchup',axis=1,inplace=True)

In [377]:
#seperate the data set into train and test data set
df_all = df_all.sample(frac=1.0, random_state=123)
df_all = df_all.reset_index(drop=True)
total = df_all.shape[0]
train_num = int(total*0.7)
train = df_all.loc[0:train_num]
train.set_index('shot_id', inplace=True)
Y_train = np.array(train['shot_made_flag'].values)
train.drop('shot_made_flag',axis=1,inplace=True)

test = df_all.loc[train_num+1:total]
test.set_index('shot_id', inplace=True)
Y_test = np.array(test['shot_made_flag'].values)
test.drop('shot_made_flag',axis=1,inplace=True)

In [378]:
# use to transform letters into calculable value
letter_num = {'A':1,'a':1,'B':2,'b':2,'C':3,'c':3,'D':4,'d':4,'E':5,'e':5,'F':6,'f':6,'G':7,'g':7,'H':8,'h':8,'I':9,
              'i':9,'J':1.1,'j':1.1,'K':1.2,'k':1.2,'L':1.3,'l':1.3,'M':1.4,'m':1.4,'N':1.5,'n':1.5,'O':1.6,'o':1.6,
              'P':1.7,'p':1.7,'Q':1.8,'q':1.8,'R':1.9,'r':1.9,'S':2.1,'s':2.1,'T':2.2,'t':2.2,'U':2.3,'u':2.3,'V':2.4,
              'v':2.4,'W':2.5,'w':2.5,'X':2.6,'x':2.6,'Y':2.7,'y':2.7,'Z':2.8,'z':2.8,'(':2.9,')':3.1,'-':3.2,'3':3.3,
              ' ': 0}

In [379]:
# a function used yo vectorise each row
def features_vec(ori_arr):
    X_vec = [[0 for j in range(len(ele))] for ele in ori_arr]
    for i, arr in enumerate(ori_arr):
        cur0 = 0
        for a in arr[0]:
            cur0 += letter_num[a]
        arr[0] = cur0/10
        
        cur1 = 0
        for b in arr[1]:
            cur1 += letter_num[b]
        arr[1] = cur1/10
        
        arr[2] = arr[2]/100
        
        cur3 = 0
        for c in str(arr[3]):
            cur3 += int(c)
        arr[3] = cur3
        
        arr[4] = arr[4]/100
        arr[5] = arr[5]/10
        arr[6] = arr[6]/10
        arr[7] = arr[7]/100
        arr_11 = arr[11].replace('-','')
        arr11 = 0
        for m in arr_11:
            arr11 += int(m)
        arr[11] = arr11
#         arr_11 = int(arr[11].replace('-',''))
        arr[12] = arr[12]/10
        arr[13] = arr[13]/10
        arr[14] = int(arr[14][0])

        if arr[15] == 'Left Side(L)':
            arr[15] = 1
        elif arr[15] == 'Left Side Center(LC)':
            arr[15] = 2
        elif arr[15] == 'Center(C)':
            arr[15] = 3
        elif arr[15] == 'Right Side Center(RC)':
            arr[15] = 4
        elif arr[15] == 'Right Side(R)':
            arr[15] = 5
        else:
            arr[15] = 0
            
        cur16 = 0
        for d in arr[16]:
            cur16 += letter_num[d]
        arr[16] = cur16/10

        if arr[17] == 'Less Than 8 ft.':
            arr[17] = 1
        elif arr[17] == '8-16 ft.':
            arr[17] = 2
        elif arr[17] == '16-24 ft.':
            arr[17] = 3
        elif arr[17] == '24+ ft.':
            arr[17] = 4
        else:
            arr[17] = 0

        arr[18] = int(arr[18].replace('-','')[4:8])/1000

        cur19 = 0
        for x in arr[19]:
            cur19 += letter_num[x]
        
        arr[19] = cur19
        for j, x in enumerate(arr):
            X_vec[i][j] = x
        
    
    transform_arr = np.array(X_vec)
    
    return transform_arr
    

Vectorise features:

In [380]:
# vectorise train and test data set
train_arr=train.values
train_vec = features_vec(train_arr)

test_arr=test.values
test_vec = features_vec(test_arr)

In [381]:
# calculate prediction accuracy by logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=1e4, solver='lbfgs', max_iter=700)
logreg.fit(train_vec, Y_train)
X_pred = logreg.predict(test_vec)
score = accuracy_score(Y_test, X_pred)
print(score)

0.6585808794915035
